In [45]:
import json
import os
import re
import glob

import pandas as pd

In [46]:
#Baseline (nl2code-0428-stability bs=50,lr=1.0e-03,end_lr=0e0,att=2)
#–no word embeddings (glove=false,upd_type=full,num_layers=4,att=0)
#–fewer self-attention layers (glove=true,upd_type=full,num_layers=2,att=0)
#–no relation-aware self-attention (glove=true,upd_type=full,num_layers=0,att=0)
#-remove some types (glove=true,upd_type=no_subtypes,num_layers=4)
#-remove all types (glove=true,upd_type=merge_types,num_layers=4)

beam_size = 1

In [50]:
prefixes = (
    '../logdirs/20190428-stability/bs=50,lr=1.0e-03,end_lr=0e0',
    '../logdirs/20190521-ablations/glove=true,upd_type=full,num_layers=0',
    '../logdirs/20190521-ablations/glove=true,upd_type=full,num_layers=2',
    '../logdirs/20190521-ablations/glove=true,upd_type=no_subtypes,num_layers=4',
    '../logdirs/20190521-ablations/glove=true,upd_type=merge_types,num_layers=4',
    '../logdirs/20190521-ablations/glove=false,upd_type=full,num_layers=4',
)
exp_names = [os.path.basename(prefix) for prefix in prefixes]
results = {}

for prefix in prefixes:
    exp_name = os.path.basename(prefix)
    results[exp_name] = results_for_exp = {}
    for att_path in sorted(glob.glob(prefix + ',att=*')):
        att_number = re.search('att=(\d+)$', att_path).group(0)
        eval_paths = glob.glob(os.path.join(att_path, 'eval-val-step*-bs{beam_size}.jsonl'.format(beam_size=beam_size)))
        for eval_path in eval_paths:
            step = int(re.search('step(\d+)', eval_path).group(1))
            scores = json.load(open(eval_path))
            results_for_exp[att_number, step] = scores
    print(prefix)

../logdirs/20190428-stability/bs=50,lr=1.0e-03,end_lr=0e0
../logdirs/20190521-ablations/glove=true,upd_type=full,num_layers=0
../logdirs/20190521-ablations/glove=true,upd_type=full,num_layers=2
../logdirs/20190521-ablations/glove=true,upd_type=no_subtypes,num_layers=4
../logdirs/20190521-ablations/glove=true,upd_type=merge_types,num_layers=4
../logdirs/20190521-ablations/glove=false,upd_type=full,num_layers=4


In [48]:
friendly_exp_names = (
    'Our method',
    'No self-attention layers',
    '2 self-attention layers',
    'Fewer relation types',
    'No relation types',
    'No pretrained word embeddings'
)

In [44]:
for exp, friendly_name in zip(exp_names, friendly_exp_names):
    rows = []
    rows.extend((exp, k, v) for k, v in results[exp].items())

    best = max(rows, key=lambda x: x[2]['total_scores']['all']['exact'])
    assert len(best[2]['per_item']) == 1034
    scores = [best[2]['total_scores'][sec]['exact'] for sec in ('easy', 'medium', 'hard', 'extra', 'all')]
    print(friendly_name, '&', ' & '.join('{:.2f}\%'.format(s * 100) for s in scores), r'\\')

Our method & 57.20\% & 44.55\% & 39.66\% & 21.18\% & 42.94\% \\
No self-attention layers & 42.40\% & 24.77\% & 22.41\% & 5.88\% & 25.53\% \\
2 self-attention layers & 53.60\% & 42.50\% & 40.80\% & 17.65\% & 40.81\% \\
Fewer relation types & 44.80\% & 30.45\% & 25.86\% & 7.65\% & 29.40\% \\
No relation types & 42.40\% & 28.86\% & 28.74\% & 7.65\% & 28.63\% \\
No pretrained word embeddings & 40.80\% & 29.09\% & 27.01\% & 5.88\% & 27.76\% \\


In [51]:
# On 2019-05-25
for exp, friendly_name in zip(exp_names, friendly_exp_names):
    rows = []
    rows.extend((exp, k, v) for k, v in results[exp].items())

    best = max(rows, key=lambda x: x[2]['total_scores']['all']['exact'])
    assert len(best[2]['per_item']) == 1034
    scores = [best[2]['total_scores'][sec]['exact'] for sec in ('easy', 'medium', 'hard', 'extra', 'all')]
    print(friendly_name, '&', ' & '.join('{:.2f}\%'.format(s * 100) for s in scores), r'\\')

Our method & 57.20\% & 44.55\% & 39.66\% & 21.18\% & 42.94\% \\
No self-attention layers & 42.00\% & 25.68\% & 22.99\% & 5.88\% & 25.92\% \\
2 self-attention layers & 56.00\% & 45.00\% & 40.23\% & 19.41\% & 42.65\% \\
Fewer relation types & 47.60\% & 30.45\% & 25.86\% & 10.00\% & 30.46\% \\
No relation types & 46.80\% & 29.55\% & 29.89\% & 8.82\% & 30.37\% \\
No pretrained word embeddings & 40.80\% & 29.09\% & 27.01\% & 5.88\% & 27.76\% \\
